##Read data from silver table

In [0]:
from pyspark.sql import functions as F

df_silver = spark.table("lakehouse.silver.sales_silver")
display(df_silver)

##Sales by Region

In [0]:
df_region = (
    df_silver
    .groupBy("region")
    .agg(
        F.sum("total_amount").alias("total_sales"),
        F.count("order_id").alias("num_orders")
    )
)

display(df_region)

#write into gold table
df_region.write.format("delta").mode("overwrite").saveAsTable("lakehouse.gold.sales_by_region")


##Daily Sales

In [0]:
df_daily = (
    df_silver
    .groupBy("order_date")
    .agg(F.sum("total_amount").alias("daily_sales"))
)

display(df_daily)

#write into gold table
df_daily.write.format("delta").mode("overwrite").saveAsTable("lakehouse.gold.daily_sales")


In [0]:
%sql
SELECT * FROM lakehouse.gold.sales_by_region ORDER BY total_sales DESC;

In [0]:
%sql
SELECT * FROM lakehouse.gold.daily_sales ORDER BY order_date;